In [ ]:
import boto3
iam = boto3.resource('iam')

In [ ]:
# すべてのユーザー
list(iam.users.all())

In [ ]:
# すべてのグループ
list(iam.groups.all())

In [ ]:
# すべてのロール
list(iam.roles.all())

In [ ]:
# すべてのポリシー ※実行には10秒ほどかかる
list(iam.policies.all())

In [ ]:
# すべてのポリシーの数 ※実行には10秒ほどかかる
len(list(iam.policies.all()))

In [ ]:
# フィルターを使用すると絞り込みを行うことができる。
# AWS 管理ポリシーの数
len(list(iam.policies.filter(Scope='AWS')))

In [ ]:
# ユーザー管理ポリシーの数
len(list(iam.policies.filter(Scope='Local')))

In [ ]:
# ユーザーの作成

iam.User('taro').create()

In [ ]:
# グループの作成

iam.Group('developers').create()

In [ ]:
# ロールの作成

# Role は、iam.Role('...').create() では作ることができない(createメソッドがない)
# 下記のように作る。

import json
policy={
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "lambda.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    }
  ]
}
iam.create_role(RoleName='lambda_role',AssumeRolePolicyDocument=json.dumps(policy))

In [ ]:
# ユーザー、グループ、ロールにポリシーを付与する例

policy_arn = 'arn:aws:iam::aws:policy/AmazonS3FullAccess'

iam.User(name='taro').attach_policy(PolicyArn=policy_arn)
iam.Group('developers').attach_policy(PolicyArn=policy_arn)
iam.Role(name='lambda_role').attach_policy(PolicyArn=policy_arn)

In [ ]:
# 「カスタマー管理ポリシー」（　「ユーザーによる管理」ポリシー）の作成

import json

policy={
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "ec2:DescribeInstances",
            "Resource": "*"
        }
    ]
}

iam.create_policy(PolicyName='ptest', PolicyDocument=json.dumps(policy))

In [ ]:
# 「カスタマー管理ポリシー」（　「ユーザーによる管理」ポリシー）のバージョンアップ

# v2, v3, v4, v5とバージョンが上がっていく。
# ポリシーあたり、バージョンはトータルで5個までしか作れない。
# たとえばv1やv2を削除すれば、v6やv7をつくることができる。

import json

policy={
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "ec2:DescribeInstances",
            "Resource": "*",
            "Condition": {
                "StringLike": {
                    "ec2:ResourceTag/Name": "WebServer*"
                }
            }
        }
    ]
}

iam.Policy(arn='arn:aws:iam::516830931548:policy/ptest').create_version(PolicyDocument=json.dumps(policy))

In [ ]:
# v2をデフォルトにする
iam.meta.client.set_default_policy_version(
    PolicyArn='arn:aws:iam::516830931548:policy/ptest',
    VersionId='v2'
)

In [ ]:
# ユーザーに「インラインポリシー」を追加する

import json

policy={
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "dynamodb:*",
            "Resource": "*"
        }
    ]
}

iam.User(name='taro').create_policy(PolicyName='dynamofull', PolicyDocument=json.dumps(policy))

In [ ]:
# ユーザーをグループに所属させる

iam.User('taro').add_group(GroupName='developers')

In [ ]:
# ユーザーがマネジメントコンソールにログインするためのパスワードを発行する
iam.User('taro').create_login_profile(Password='taropass')

In [ ]:
# ユーザーのアクセスキーID、シークレットアクセスキーを発行する

# （アクセスキーID、シークレットアクセスキー）のペアは2組まで作ることができる。
#3組目を作る場合は、すでに存在する組を削除しなければならない。

iam.User('taro').create_access_key_pair()